In [231]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
import os
import random
from statsmodels.stats.proportion import binom_test
from statsmodels.stats.multitest import multipletests

def parse_genome(df):
    genome_id = df['#query'][0].split('_')[0]
    keggs = df['KEGG_ko'].replace('-', None).dropna()
    keggs = list(map(lambda x: x.split(','), keggs.values))
    keggs = sum(keggs, [])
    keggs = pd.DataFrame({'KEGG_ko': keggs})
    keggs['genome_id'] = genome_id
    return keggs

def to_sparse_matrix(func_df, genome_id='genome_id', kegg_id='KEGG_ko'):
    # create genome-specific index
    ogus = list(set(func_df[genome_id]))
    ogu_lookup = pd.Series(np.arange(0, len(ogus)), ogus)
    # create KEGG-specific index
    keggs = list(set(func_df[kegg_id]))
    kegg_lookup = pd.Series(np.arange(0, len(keggs)), keggs)
    # rename names as numbers
    ogu_id = func_df[genome_id].apply(lambda x: ogu_lookup.loc[x]).astype(np.int64)
    kegg_id = func_df[kegg_id].apply(lambda x: kegg_lookup.loc[x]).astype(np.int64)
    # assign the presence / absence of a gene
    func_df['count'] = 1
    c = func_df['count'].values
    # format into a matrix
    data = coo_matrix((c, (ogu_id, kegg_id)))
    ko_ogu = pd.DataFrame(data.todense(), index=ogus, columns=keggs)
    return ko_ogu

def btest(pa1, pa2, seed=0, return_proportions=False):
    """ Performs genome wide binomial test between two groups of taxa
    Parameters
    ----------
    df1 : pd.DataFrame
        Rows are taxa, columns are genes
    df2 : pd.DataFrame
        Rows are taxa, columns are genes
    Returns
    -------
    pd.Series : list of genes associated with df1
    pd.Series : list of genes associated with df2
    """
    np.random.seed(seed)
    random.seed(seed)
    #pa1 = df1 > 0
    #pa2 = df2 > 0
    idx = list(set(pa1.columns) | set(pa2.columns))
    idx.sort()
    pa1 = pa1.sum(axis=0).reindex(idx).fillna(0)
    pa2 = pa2.sum(axis=0).reindex(idx).fillna(0)
    n = pa1 + pa2
    obs = list(zip(list(pa1.values), list((pa2.values + 1) / (pa2 + 1).sum()), list(n.values)))
    pvals = pd.Series([binom_test(a, n, b, 'two-sided') for (a, b, n) in obs],
                      index=n.index)
    if return_proportions:
        res = pd.DataFrame({'groupA': pa1, 'groupB': pa2, 'pval': pvals})
        def relabel_f(x):
            if x['groupA'] < x['groupB']:
                return 'groupB'
            else:
                return 'groupA'
        res['side'] = res.apply(relabel_f, axis=1)
        return res

    return pvals

def log_pvalue(lr, alpha=0.1, filter=True):
    """ Converts pvalues to -log(pvalue)
    Also performs Boniferroni correction.
    """
    lr = lr.reset_index()
    # lr.columns = ['KEGG', 'pvalue']
    lr['-log(pvalue)'] = -np.log(lr['pvalue'] + 1e-200)
    res = multipletests(lr['pvalue'], method='fdr_bh', alpha=alpha)
    lr['pvalue_corrected'] = res[1]
    if filter:
        lr = lr.loc[res[0]]
        return lr
    return lr

In [47]:
#run only once to generate two columns file metadata_new 
# #parse genomes-all_metadata.tsv: get species names from Lineage and Species_rep
# metadata = pd.read_table('../../../../Downloads/genomes-all_metadata.tsv')
# metadata_new = metadata[['Lineage','Species_rep']]
# metadata_new['species'] = metadata_new['Lineage'].str.split(';').str[6]
# metadata_new['Species'] = metadata_new['species'].str.split('__').str[1]
# metadata_new = metadata_new[['Species','Species_rep']]
# metadata_new.to_csv('../table/eggNOG_species_rep.txt', sep = '\t', index = False)

In [95]:
metadata_new = pd.read_table('../table/eggNOG_species_rep.txt')

In [140]:
#get the species_rep for specific species
#  'CAG-170 sp002404795',
#  'CAG-83 sp003487665',
#  'UBA11452 sp003526375',
#  'Victivallis vadensis'
asd_cd_positive = ['CAG-170 sp002404795',
 'CAG-83 sp003487665',
 'UBA11452 sp003526375',
 'Victivallis vadensis']

In [141]:
type(asd_cd_positive)

list

In [139]:
#load tables
table_t2d = pd.read_table('../table/QinT2D_deseq2_all.txt')
table_cd = pd.read_table('../table/Franzosa_CD_deseq2_all.txt')
table_asd = pd.read_table('../table/Dan_n_Wang_ASD_deseq2_all.txt')

#negative: more abundant in cases
table_t2d_negative = table_t2d.sort_values(by=['CI_95'],ascending=True).head(100)
table_cd_negative = table_cd.sort_values(by=['CI_95'],ascending=True).head(100)
table_asd_negative = table_asd.sort_values(by=['CI_95'],ascending=True).head(100)

#positive: more abundant in controls
table_t2d_positive = table_t2d.sort_values(by=['CI_5'],ascending=False).head(100)
table_cd_positive = table_cd.sort_values(by=['CI_5'],ascending=False).head(100)
table_asd_positive = table_asd.sort_values(by=['CI_5'],ascending=False).head(100)


In [145]:
t2d_positive = list(table_t2d_positive['Unnamed: 0'])
t2d_negative = list(table_t2d_negative['Unnamed: 0'])

In [168]:
asd_cd_positive_rep = metadata_new[metadata_new['Species'].isin(asd_cd_positive)]
Species_rep_ids = asd_cd_positive_rep['Species_rep'].drop_duplicates()
t2d_positive_rep = metadata_new[metadata_new['Species'].isin(t2d_positive)]
Species_rep_ids_t2d_positive = t2d_positive_rep['Species_rep'].drop_duplicates()
t2d_negative_rep = metadata_new[metadata_new['Species'].isin(t2d_negative)]
Species_rep_ids_t2d_negative = t2d_negative_rep['Species_rep'].drop_duplicates()

In [175]:
Species_rep_ids_t2d_positive
Species_rep_ids_t2d_negative
#why length = 101?

6       MGYG000000007
7       MGYG000000008
20      MGYG000000021
45      MGYG000000046
79      MGYG000000080
            ...      
4395    MGYG000004424
4588    MGYG000004618
4775    MGYG000004805
4791    MGYG000004821
4831    MGYG000004861
Name: Species_rep, Length: 101, dtype: object

In [165]:
eggNOG_dir

'http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v2.0/species_catalogue'

In [177]:
mkdir T2D_negative

In [255]:
for i in Species_rep_ids_t2d_negative:
    os.system("wget '{}/{}/{}/genome/{}_eggNOG.tsv' -O T2D_negative/{}_eggNOG.tsv".format(eggNOG_dir, i[:-2], i, i, i))

In [166]:
mkdir T2D_positive

In [256]:
#download the eggNOG tsv files
for i in Species_rep_ids_t2d_positive:
    os.system("wget '{}/{}/{}/genome/{}_eggNOG.tsv' -O T2D_positive/{}_eggNOG.tsv".format(eggNOG_dir, i[:-2], i, i, i))

In [179]:
# df1 = pd.read_table('../../../../Downloads/MGYG000000001_eggNOG.tsv')
# df2 = pd.read_table('../../../../Downloads/MGYG000002834_eggNOG.tsv')

In [207]:
df_list_p = []
for i in Species_rep_ids_t2d_positive:
    f_name = 'T2D_positive/{}_eggNOG.tsv'.format(i)
    df_parsed = parse_genome(pd.read_table(f_name))
    df_list_p.append(df_parsed)
    
df_cat_p = pd.concat(df_list_p, axis=0)
genome_kegg_counts_p = to_sparse_matrix(df_cat_p)


df_list_n = []
for i in Species_rep_ids_t2d_negative:
    f_name = 'T2D_negative/{}_eggNOG.tsv'.format(i)
    df_parsed = parse_genome(pd.read_table(f_name))
    df_list_n.append(df_parsed)
    
df_cat_n = pd.concat(df_list_n, axis=0)
genome_kegg_counts_n = to_sparse_matrix(df_cat_n)

In [254]:
!rm -r T2D_negative/
!rm -r T2D_positive/

In [224]:
#compare controls and cases: for top 100 microbes
kegg_t2d = btest(genome_kegg_counts_p, genome_kegg_counts_n, return_proportions=False)
kegg_t2d = pd.DataFrame({'pvalue': kegg_t2d})
kegg_t2d = kegg_t2d.reset_index().rename(columns={'index': 'KEGG'})
kegg_t2d = log_pvalue(kegg_t2d, alpha=0.1)

In [249]:
kegg_t2d.sort_values(by = ['pvalue'])

index    pvalue  -log(pvalue)  pvalue_corrected
1155  ko:K02078  0.000000    460.517019          0.000000
2786  ko:K06921  0.000000    460.517019          0.000000
397   ko:K00791  0.000000    460.517019          0.000000
1221  ko:K02238  0.000000    460.517019          0.000000
2859  ko:K07011  0.000000    460.517019          0.000000
...         ...       ...           ...               ...
1897  ko:K03492  0.023240      3.761895          0.032526
3713  ko:K09963  0.024038      3.728110          0.033636
3332  ko:K08161  0.028225      3.567542          0.039484
3606  ko:K09773  0.029101      3.536978          0.040700
5084  ko:K18926  0.037555      3.281940          0.052511

[4038 rows x 4 columns]

In [252]:
kegg_t2d.sort_values(by = ['pvalue']).head(50)

index  pvalue  -log(pvalue)  pvalue_corrected
1155  ko:K02078     0.0    460.517019               0.0
2786  ko:K06921     0.0    460.517019               0.0
397   ko:K00791     0.0    460.517019               0.0
1221  ko:K02238     0.0    460.517019               0.0
2859  ko:K07011     0.0    460.517019               0.0
2868  ko:K07025     0.0    460.517019               0.0
2913  ko:K07085     0.0    460.517019               0.0
394   ko:K00788     0.0    460.517019               0.0
2922  ko:K07098     0.0    460.517019               0.0
579   ko:K01091     0.0    460.517019               0.0
2931  ko:K07107     0.0    460.517019               0.0
1111  ko:K02030     0.0    460.517019               0.0
1098  ko:K02016     0.0    460.517019               0.0
1097  ko:K02015     0.0    460.517019               0.0
1096  ko:K02014     0.0    460.517019               0.0
1095  ko:K02013     0.0    460.517019               0.0
2945  ko:K07126     0.0    460.517019               0.0
2949  ko:K07133     0.0    460.517019               0.0
1087  ko:K02005     0.0    460.517019               0.0
4657  ko:K15923     0.0    460.517019               0.0
1271  ko:K02355     0.0    460.517019               0.0
2742  ko:K06871     0.0    460.517019               0.0
1304  ko:K02406     0.0    460.517019               0.0
4102  ko:K12132     0.0    460.517019               0.0
1615  ko:K02945     0.0    460.517019               0.0
511   ko:K00970     0.0    460.517019               0.0
512   ko:K00971     0.0    460.517019               0.0
2382  ko:K05349     0.0    460.517019               0.0
2390  ko:K05366     0.0    460.517019               0.0
158   ko:K00266     0.0    460.517019               0.0
4080  ko:K11991     0.0    460.517019               0.0
2562  ko:K06131     0.0    460.517019               0.0
2567  ko:K06142     0.0    460.517019               0.0
2571  ko:K06147     0.0    460.517019               0.0
2578  ko:K06158     0.0    460.517019               0.0
2588  ko:K06173     0.0    460.517019               0.0
2595  ko:K06180     0.0    460.517019               0.0
1400  ko:K02529     0.0    460.517019               0.0
527   ko:K00991     0.0    460.517019               0.0
1353  ko:K02470     0.0    460.517019               0.0
1327  ko:K02429     0.0    460.517019               0.0
1326  ko:K02428     0.0    460.517019               0.0
1086  ko:K02004     0.0    460.517019               0.0
1085  ko:K02003     0.0    460.517019               0.0
1074  ko:K01992     0.0    460.517019               0.0
1072  ko:K01990     0.0    460.517019               0.0
3173  ko:K07568     0.0    460.517019               0.0
336   ko:K00655     0.0    460.517019               0.0
3635  ko:K09808     0.0    460.517019               0.0
3192  ko:K07636     0.0    460.517019               0.0